## Defining `ManZoneTransformer` Class

In [ ]:
import torch
import torch.nn as nn

class ManZoneTransformer(nn.Module):

  def __init__(self, feature_len=5, model_dim=64, num_heads=2, num_layers=4, dim_feedforward=256, dropout=0.1, output_dim=2):
      super(ManZoneTransformer, self).__init__()
      self.feature_norm_layer = nn.BatchNorm1d(feature_len)

      self.feature_embedding_layer = nn.Sequential(
          nn.Linear(feature_len, model_dim),
          nn.ReLU(),
          nn.LayerNorm(model_dim),
          nn.Dropout(dropout),
      )

      transformer_encoder_layer = nn.TransformerEncoderLayer(
          d_model=model_dim,
          nhead=num_heads,
          dim_feedforward=dim_feedforward,
          dropout=dropout,
          batch_first=True,
      )
      self.transformer_encoder = nn.TransformerEncoder(transformer_encoder_layer, num_layers=num_layers)

      self.player_pooling_layer = nn.AdaptiveAvgPool1d(1)

      self.decoder = nn.Sequential(
          nn.Linear(model_dim, model_dim),
          nn.ReLU(),
          nn.Dropout(dropout),
          nn.Linear(model_dim, model_dim // 4),
          nn.ReLU(),
          nn.LayerNorm(model_dim // 4),
          nn.Linear(model_dim // 4, output_dim),
      )

  def forward(self, x):
      # x shape: (batch_size, num_players, feature_len)
      x = self.feature_norm_layer(x.permute(0, 2, 1)).permute(0, 2, 1)
      x = self.feature_embedding_layer(x)
      x = self.transformer_encoder(x)
      x = self.player_pooling_layer(x.permute(0, 2, 1)).squeeze(-1)
      x = self.decoder(x)
      return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ManZoneTransformer(
    feature_len=5,    # num of input features (x, y, v_x, v_y, defense)
    model_dim=64,     # experimented with 96 & 128... seems best
    num_heads=2,      # 2 seems best (but may have overfit when tried 4... may be worth iterating)
    num_layers=4,
    dim_feedforward=64 * 4,
    dropout=0.1,      # 10% dropout to prevent overfitting... iterate as model becomes more complex (industry std is higher, i believe)
    output_dim=2      # man or zone classification
).to(device)

## Making Preds (Out of Sample)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.optim import AdamW
pd.options.mode.chained_assignment = None
import warnings
import random

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# reading static CSV files (currently in GDrive)
games = pd.read_csv("/content/drive/MyDrive/nfl-big-data-bowl-2025/games.csv")
player_play = pd.read_csv("/content/drive/MyDrive/nfl-big-data-bowl-2025/player_play.csv")
players = pd.read_csv("/content/drive/MyDrive/nfl-big-data-bowl-2025/players.csv")
plays = pd.read_csv("/content/drive/MyDrive/nfl-big-data-bowl-2025/plays.csv")

all_weeks = []

for week_number in range(1, 10):
  week_data = process_week_data_preds(week_number, plays)
  all_weeks.append(week_data)

all_tracking = pd.concat(all_weeks, ignore_index=True)
all_tracking = all_tracking[(all_tracking['club'] != 'football') & (all_tracking['passAttempt'] == 1)]

Finished reading Week 1 data
Finished processing Week 1 data

Finished reading Week 2 data
Finished processing Week 2 data

Finished reading Week 3 data
Finished processing Week 3 data

Finished reading Week 4 data
Finished processing Week 4 data

Finished reading Week 5 data
Finished processing Week 5 data

Finished reading Week 6 data
Finished processing Week 6 data

Finished reading Week 7 data
Finished processing Week 7 data

Finished reading Week 8 data
Finished processing Week 8 data

Finished reading Week 9 data
Finished processing Week 9 data



In [ ]:
# ~20mins per week
## -- many ways to optimize (currently isn't batched)

import polars as pl

for week_eval in range(1, 10):

  tracking_df = all_tracking[all_tracking['week'] == week_eval]
  tracking_df_polars = pl.DataFrame(tracking_df)  # convert to Polars

  list_ids = list(set(tracking_df['frameUniqueId']))
  best_model_path = f"/content/drive/MyDrive/nfl-big-data-bowl-2025/best_model_week{week_eval}.pth"
  model.load_state_dict(torch.load(best_model_path, weights_only=True))

  model.eval()

  results = []

  with warnings.catch_warnings():

      warnings.simplefilter("ignore", category=DeprecationWarning)
      print(f"Starting loop for week {week_eval}...")

      for idx, frame_id in enumerate(list_ids, start=1):  # enumerating to print out certain invervals

          if idx % 20000 == 0:
            print(f"Processed {idx} frame_ids for week {week_eval}...")
            print(f"{idx / len(list_ids):.2f}%")

          play_id = "_".join(frame_id.split("_")[:2])
          frame_num = frame_id.split("_")[-1]
          frame_num = int(frame_num)

          frame = tracking_df_polars.filter(pl.col("frameUniqueId") == frame_id)
          frame = frame.to_pandas()

          frame_tensor = prepare_tensor(frame)
          frame_tensor = frame_tensor.to(device)  # Move to device if necessary

          with torch.no_grad():
              outputs = model(frame_tensor)  # Shape: [num_frames, num_classes]
              probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

              zone_prob = probabilities[0][0]
              man_prob = probabilities[0][1]

              pred = 0 if zone_prob > man_prob else 1
              actual = frame['pff_manZone'].iloc[0]

              results.append({
                  'frameUniqueId': frame_id,
                  'uniqueId': play_id,
                  'frameId': frame_num,
                  'zone_prob': zone_prob,
                  'man_prob': man_prob,
                  'pred': pred,
                  'actual': actual
              })

      week_results = pd.DataFrame(results)
      week_results.to_csv(f"/content/drive/MyDrive/nfl-big-data-bowl-2025/week{week_eval}_preds.csv")
      print(f"Finished week {week_eval}... saving to week{week_eval}_preds.csv")
      print()

Starting loop for week 4...
Processed 20000 frame_ids for week 4...
0.16%
Processed 40000 frame_ids for week 4...
0.31%
Processed 60000 frame_ids for week 4...
0.47%
Processed 80000 frame_ids for week 4...
0.63%
Processed 100000 frame_ids for week 4...
0.78%
Processed 120000 frame_ids for week 4...
0.94%
Finished week 4... saving to week4_preds.csv

Starting loop for week 5...
Processed 20000 frame_ids for week 5...
0.14%
Processed 40000 frame_ids for week 5...
0.29%
Processed 60000 frame_ids for week 5...
0.43%
Processed 80000 frame_ids for week 5...
0.58%
Processed 100000 frame_ids for week 5...
0.72%
Processed 120000 frame_ids for week 5...
0.86%
Finished week 5... saving to week5_preds.csv

Starting loop for week 6...
Processed 20000 frame_ids for week 6...
0.16%
Processed 40000 frame_ids for week 6...
0.32%
Processed 60000 frame_ids for week 6...
0.49%
Processed 80000 frame_ids for week 6...
0.65%
Processed 100000 frame_ids for week 6...
0.81%
Processed 120000 frame_ids for week 6

In [ ]:
for week_eval in range(1,10):

  week_df = all_tracking[all_tracking['week'] == week_eval]
  preds_week = pd.read_csv(f"/content/drive/MyDrive/nfl-big-data-bowl-2025/week{week_eval}_preds.csv")

  preds_week = preds_week[['frameUniqueId', 'zone_prob', 'man_prob', 'pred', 'actual']]

  tracking_preds = week_df.merge(preds_week, on='frameUniqueId',how='left')

  tracking_preds.to_csv(f"/content/drive/MyDrive/nfl-big-data-bowl-2025/tracking_week_{week_eval}_preds.csv")

Next:

*   New notebook for merging these results with (x,y) tracking data for each frame
*   Get a line graph of how it performs
* Optimize bayes, ema, etc. accordingly to get highest accuracy possible in the presnap (did it before, but iterate again)

Next (pt. 2)

* lose bayes, ema, etc. & iterate again to find best way to represent these frame-by-frame... don't want overly smooth or overly sporadic graphs (this will be manual i think)

* further refine methodology for tells, make it iteratable, meaning can change the parameters

Next (pt. 3)

* "draw" dots of slot cb, outside cb, etc. (almost like a nearest-neighbors algorithim) to neatly group where a player was

* split by motion & non-motion!

* for motion group: classify/label the different motions (idk how)

Next (pt. 4)

* draw & calculate each player's path from the slot/outside,etc. & what their tell one

Final vis (pt. 5)

* player-specific: over these 9 weeks what were andre cisco's tells?

* team-specific: over these 9 weeks what tells did the jaguars have? can we map that out from base-formation?

* report: some combination of these few... pair with frontend